### important: whenever you make changes to an imported notebook you need to select "restart the kernel" in this notebook to load the changes then rerun the cells

In [ ]:
# parameters
TIMER = True
REFERENCE_MISSION = 2
mission_numbers = [REFERENCE_MISSION, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]

BBOX_COLOR = (0, 0, 255)
DEBUG_IMAGES = True
NUM_BBOX_ITERATIONS = 2

In [ ]:
import import_ipynb
import KyleFunctions as KF
import AlexFunctions as AF
import JoshFunctions as JF
import LoadData as LD
import VisualizeResults as VR

import time
import numpy as np

# Load Data

In [ ]:
start = time.time()

# load all of the images and regions (bgr, hsv, histogram)
data = LD.run_load_data(mission_numbers, debug_images=False, save_images=False)

# original color images
mission_images_bgr = data[0]
grass_images_bgr = data[1]
building_images_bgr = data[2]

# histogram normalized color images
histogram_mission_images_bgr = data[3]
histogram_grass_images_bgr = data[4]
histogram_building_images_bgr = data[5]

# original hsv images
mission_images_hsv = data[6]
grass_images_hsv = data[7]
building_images_hsv = data[8]

# histogram normalized hsv images
histogram_mission_images_hsv = data[9]
histogram_grass_images_hsv = data[10]
histogram_building_images_hsv = data[11]

if TIMER: print('Duration: %.3f sec' % (time.time()-start))

# Anomaly Detection

In [ ]:
start = time.time()

# Kyle - anomaly detection


# Alex - anomaly detection
anomaly_masks_1 = AF.anomaly_detection_1(mission_numbers, mission_images_hsv, histogram_mission_images_hsv, debug_images=False)

if TIMER: print('Duration: %.3f sec' % (time.time()-start))

# Grass Region Heatmap

In [ ]:
start = time.time()

# Kyle - heatmap
# import import_ipynb
# import numpy as np
# import KyleFunctions as KF # TODO code needs to be put into function format
# import KyleFunctions as KF
# import LoadData as LD
# import VisualizeResults as VR

heatmap,percents = KF.score_missions(histogram_grass_images_bgr, 10)
# print(percents)
# AF.plot_images(image_list=heatmap)
# for each in percents:
#     print(np.sum(each))

if TIMER: print('Duration: %.3f sec' % (time.time()-start))

# Roof

In [ ]:
start = time.time()

# Josh - roof

roof_masks, roof_anomaly_percentages = JF.findRoofAnomalies(histogram_building_images_hsv)
AF.plot_images(roof_masks, ['roof_masks ' + str(mission_number) for mission_number in mission_numbers])
print(roof_anomaly_percentages)

if TIMER: print('Duration: %.3f sec' % (time.time()-start))

# Visualize Output

In [ ]:
start = time.time()

# combine the bounding boxes from anomaly detection
# combine the heatmap and roof mask
# display histograms, plots, metrics, etc

# TODO add all object masks to [anomaly_masks_1, ...]
VR.run_visualize_results(mission_numbers, [anomaly_masks_1], histogram_mission_images_bgr, NUM_BBOX_ITERATIONS, BBOX_COLOR, heatmap, percents, roof_masks, roof_anomaly_percentages, debug_images=False)

if TIMER: print('Duration: %.3f sec' % (time.time()-start))

# Generate Summary Output File

In [ ]:
from fpdf import FPDF

class PDF(FPDF):
    def footer(self): # automatically called by add_page()
        self.set_y(-30)
        self.set_font('Arial', '', 8)
        self.cell(0, 10, 'Dronezz - %s' % self.page_no(), 0, 0, 'R')

def display_header(text='', align='L'):
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(w=0, h=24, txt=text, border=0, ln=1, align=align, fill=False)
    
def display_text(text='', align='L'):
    pdf.set_font('Arial', '', 8)
    pdf.multi_cell(w=0, h=16, txt=text, border=0, align=align, fill=False)

# ----------------------------------------

pdf = PDF(orientation='Portrait', unit='pt', format='Letter')
pdf.add_page()

# ----------------------------------------

pdf.set_font('Arial', 'B', 24)
text = 'Dronezz'
pdf.cell(w=0, h=30, txt=text, border=0, ln=1, align='C', fill=False)

pdf.set_font('Arial', 'B', 16)
text = 'description'
pdf.cell(w=0, h=30, txt=text, border=0, ln=1, align='C', fill=False)

text = 'Alex Faché, Mason Heim, Josh Moore, Kyle Turner\nGeorgia Institute of Technology - School of Electrical and Computer Engineering\nECE 4011 Spring 2020, ECE 4873 Fall 2020'
display_text(text, align='C')

# ----------------------------------------

text = 'Data'
display_header(text)

text = 'The images for this project were collected using a commerical Parrot Anafi quadcopter, utilizing the onboard color camera and Pix4Dcapture missiong planning. Each image was taken at an altitude of 40m resulting ' \
    'in a ground resultion of 1.34 cm/pixel. The area in question spans 79x82m resulting in 54 overlapping images that were combined to form a single image using Pix4Dmapper'
display_text(text)

# ----------------------------------------

text = 'Histogram Normalized'
display_header(text)

text = 'Histogram normalization was applied to best remove the variation in brightness due to the Sun and clouds. Mission 2 was used as the reference mission for each normalization.'
display_text(text)

pdf.image(name='..\\report_images\\1.png', w=100, h=100, type='PNG')

# ----------------------------------------

pdf.add_page()

text = 'Bounding Boxes'
display_header(text)

text = 'text'
display_text(text)

# ----------------------------------------

text = 'Anomaly Detection'
display_header(text)

text = 'text'
display_text(text)

# ----------------------------------------

text = 'Heatmaps'
display_header(text)

text = 'text'
display_text(text)

# ----------------------------------------

text = 'Results'
display_header(text)

text = 'text'
display_text(text)

# ----------------------------------------

pdf.output('auto_generated_report.pdf', 'F')